<a href="https://colab.research.google.com/github/CauchyLee1981/colab/blob/master/wordToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [33]:
cd drive/My\ Drive/Colab\ Notebooks

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks'
/content/drive/My Drive/Colab Notebooks


# 案１のデータGenerator

In [0]:
import os
import numpy as np
from keras.utils import np_utils


def my_generator(path="data", batch_size=50, n_classes=2, frame_num=32, shift=8):
    key_point=25
    channel=3
    
    if not os.path.exists(path):
        print(path, "is not exist\n")
        return

    train_data = []
    counter = 0
    for root, folders, _ in os.walk(path): break
    print(root,folders)
    while True:
      for folder in folders:
        for root2, _, files in os.walk(os.path.join(root, folder)): break
        print("\n",root2,files)
        for file in files:
          with open(os.path.join(root2, file)) as tmp_file:
            a = tmp_file.read()
            b = np.array(a.split()).reshape(-1, key_point, channel).astype(np.float32)
            for i in range(5):
              start = i * shift +frame_num
              end = start + frame_num
              X = b[start:end]
              #print(X.size)
              if not X.size == frame_num*key_point*channel:
                continue
              X1 = X.copy()
              X1[:-1, :, :] = X1[1:, :, :] - X1[:-1, :, :]
              X = X.reshape(1, frame_num, key_point, channel)
              X1 = X1.reshape(1, frame_num, key_point, channel)
              y = np_utils.to_categorical(int(folder), num_classes=n_classes)
              y = y.reshape(1, len(y))
              yield [X,X1], y



gen = my_generator(n_classes=100)
i = 0

for X,y in gen:
  #print(X)
  print(y)
  if i >= 50: 
    break
  else:
    i=i+1


# 案２のGenerator

## file list generator

In [0]:
import os
import numpy as np
from keras.utils import np_utils


def my_generator(path="data"):
    
    if not os.path.exists(path):
        print(path, "is not exist\n")
        return

    train_data = []
    counter = 0
    for root, folders, _ in os.walk(path): break
    print(root,folders)
    
    with open("listfile/SLR1_list_train.txt", mode='w') as train_f:
      with open("listfile/SLR1_list_test.txt", mode='w') as test_f:
        for folder in folders:
          for root2, _, files in os.walk(os.path.join(root, folder)): break
          print("\n",root2,files)
          for file in files:
            temp_str = folder + "," + os.path.join(root2, file) + "\n"
            # ファイル名に第三Underbarの後ろの数字を取得
            testflg=file.split('.',1)[0].split('_')[-1]
            print(testflg)
            if testflg is '4':
              test_f.write(temp_str)
            else:
              train_f.write(temp_str)


my_generator()


file list よりデータをgenerateする。

In [0]:
import os
import numpy as np
import random
from keras.utils import np_utils


def my_generator_1(list_path="listfile/SLR1_list_train.txt", batch_size=50, n_classes=2, frame_num=32, shift=8):
  key_point=25
  channel=3
  
  if not os.path.exists(list_path):
      print(path, "is not exist\n")
      return

  train_data = []
  counter = 0

  while True:
    with open(list_path) as list_f:
      files_list = list_f.readlines()
      random.shuffle(files_list)
      for line in files_list:
        #print(line)
        folder, file = line.split('\n')[0].split(',')
        #print(folder,file)
        with open(file) as tmp_file:
          a = tmp_file.read()
          b = np.array(a.split()).reshape(-1, key_point, channel).astype(np.float32)
          for i in range(5):
            start = i * shift +frame_num
            end = start + frame_num
            X = b[start:end]
            #print(X.size)
            if not X.size == frame_num*key_point*channel:
              continue
            X1 = X.copy()
            X1[:-1, :, :] = X1[1:, :, :] - X1[:-1, :, :]
            X = X.reshape(1, frame_num, key_point, channel)
            X1 = X1.reshape(1, frame_num, key_point, channel)
            y = np_utils.to_categorical(int(folder), num_classes=n_classes)
            y = y.reshape(1, len(y))
            yield [X,X1], y


gen = my_generator_1(n_classes=100)
i = 0

for X,y in gen:
  #print(X)
  print(y)
  if i >= 50: 
    break
  else:
    i=i+1

# 一括データ作成

In [0]:
import os
import numpy as np
import random
from keras.utils import np_utils
import time

def make_data(list_path="listfile/SLR1_list_train.txt", batch_size=50, n_classes=2, frame_num=32, shift=8, train=True):
  key_point=25
  channel=3
  
  if not os.path.exists(list_path):
      print(path, "is not exist\n")
      return

  train_data_X = []
  train_data_X1 = []
  train_data_y = []

  counter = 0

  with open(list_path) as list_f:
    files_list = list_f.readlines()
    if train:
      random.shuffle(files_list)
    for line in files_list:
      #print(line)
      folder, file = line.split('\n')[0].split(',')
      #print(folder,file)
      with open(file) as tmp_file:
        a = tmp_file.read()
        b = np.array(a.split()).reshape(-1, key_point, channel).astype(np.float32)
        for i in range(5):
          start = i * shift +frame_num
          end = start + frame_num
          X = b[start:end]
          #print(X.size)
          if not X.size == frame_num*key_point*channel:
            continue
          X1 = X.copy()
          X1[:-1, :, :] = X1[1:, :, :] - X1[:-1, :, :]
          #X = X.reshape(1, frame_num, key_point, channel)
          #X1 = X1.reshape(1, frame_num, key_point, channel)
          X = X.reshape(frame_num, key_point, channel)
          X1 = X1.reshape(frame_num, key_point, channel)
          y = np_utils.to_categorical(int(folder), num_classes=n_classes)
          #y = y.reshape(1, len(y))
          
          train_data_X.append(X)
          train_data_X1.append(X1)
          train_data_y.append(y)
          counter += 1
          #if counter > 50:
          #  break
      #if counter > 50:
      #  break


  train_data_X = np.array(train_data_X)
  train_data_X1 = np.array(train_data_X1)
  train_data_y = np.array(train_data_y)
  return train_data_X, train_data_X1, train_data_y




In [0]:
t1 = time.time() 
X, X1, y = make_data(n_classes=100)
t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

In [0]:
print(X[0].shape)
print(X1[0].shape)
print(y[0].shape)


In [0]:
print(y[0])

# ネットの定義

In [80]:
import time

import argparse
import json
import os

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import *
from keras.layers.convolutional import *
from keras.layers.core import *
from keras.models import Model
from keras.optimizers import adam


def one_obj(frame_l=32, joint_n=25, joint_d=3, words_size=500):
    input_joints = Input(name='joints', shape=(frame_l, joint_n, joint_d))
    input_joints_diff = Input(name='joints_diff', shape=(frame_l, joint_n, joint_d))

    x = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # @LeakyReLU()(x)

    x = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)

    x = Permute((1, 3, 2))(x)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = LeakyReLU()(x)  # BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x_d = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints_diff)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Permute((1, 3, 2))(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)
    x_d = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)

    x = concatenate([x, x_d], axis=-1)

    x = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)

    x = Conv2D(filters=256, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Flatten()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = Dense(words_size, activation='softmax')(x)
    return Model([input_joints, input_joints_diff], x)


model = one_obj(32, 25, 3, 100)
opt = adam(0.005, epsilon=1e-8)
model.compile(opt, 'categorical_crossentropy', metrics=['acc'])
model.summary()

early = EarlyStopping(monitor="loss", mode="min", patience=10)
lr_change = ReduceLROnPlateau(monitor="loss", verbose=1, factor=0.2, patience=100, min_lr=1e-8, cooldown=100)
lr_change.set_model(model)
lr_change.on_train_begin()
best_acc = 0.0







Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
joints_diff (InputLayer)        (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
joints (InputLayer)             (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 32, 25, 64)   256         joints_diff[0][0]                
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 25, 64)   256         joints[0][0]                     
____________________________________________________________________________________________

# Train

In [38]:
# データGeneratorで学習させている場合、なか問題がある。このあとは一括で学習させる場合、問題なさそうです。
train_generator = my_generator_1(n_classes=100)

# Train model on dataset
model.fit_generator(generator=train_generator,
          steps_per_epoch=256,
          epochs=1000)

Epoch 1/1000
256/256 [==============================] - 23s 91ms/step - loss: 15.8178 - acc: 0.0156
Epoch 2/1000
256/256 [==============================] - 23s 92ms/step - loss: 15.8663 - acc: 0.0156
Epoch 3/1000
256/256 [==============================] - 22s 87ms/step - loss: 15.9922 - acc: 0.0078
Epoch 4/1000
256/256 [==============================] - 39s 152ms/step - loss: 15.8663 - acc: 0.0156
Epoch 5/1000
256/256 [==============================] - 23s 90ms/step - loss: 15.8663 - acc: 0.0156
Epoch 6/1000
256/256 [==============================] - 22s 86ms/step - loss: 15.4885 - acc: 0.0391
Epoch 7/1000
256/256 [==============================] - 22s 87ms/step - loss: 15.7403 - acc: 0.0234
Epoch 8/1000
256/256 [==============================] - 25s 97ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 9/1000
256/256 [==============================] - 19s 76ms/step - loss: 15.8033 - acc: 0.0195
Epoch 10/1000
256/256 [==============================] - 22s 84ms/step - loss: 16.1181 - acc: 0

KeyboardInterrupt: ignored

## 下記Batch Size（128）は最高85%結果です。

In [68]:
# Train model on dataset
model.fit(x=[X,X1], y=y, epochs=200, batch_size=128,validation_split=0.25)

Train on 60772 samples, validate on 20258 samples
Epoch 1/200
60772/60772 [==============================] - 48s 789us/step - loss: 2.9367 - acc: 0.3028 - val_loss: 2.1953 - val_acc: 0.4469
Epoch 2/200
60772/60772 [==============================] - 44s 722us/step - loss: 1.1642 - acc: 0.6344 - val_loss: 1.3143 - val_acc: 0.6123
Epoch 3/200
60772/60772 [==============================] - 44s 723us/step - loss: 0.8271 - acc: 0.7284 - val_loss: 1.3190 - val_acc: 0.6114
Epoch 4/200
60772/60772 [==============================] - 44s 726us/step - loss: 0.6790 - acc: 0.7719 - val_loss: 1.0760 - val_acc: 0.6680
Epoch 5/200
60772/60772 [==============================] - 44s 721us/step - loss: 0.5851 - acc: 0.7997 - val_loss: 1.0465 - val_acc: 0.6799
Epoch 6/200
60772/60772 [==============================] - 44s 720us/step - loss: 0.5210 - acc: 0.8210 - val_loss: 1.4508 - val_acc: 0.5767
Epoch 7/200
60772/60772 [==============================] - 44s 723us/step - loss: 0.4720 - acc: 0.8368 - val_l

KeyboardInterrupt: ignored

## 下記Batch Size減少（６４）は最高86%結果です。100epoch以降過学習のようです。

In [81]:
model.fit(x=[X,X1], y=y, epochs=200, batch_size=64,validation_split=0.25)

Train on 60772 samples, validate on 20258 samples
Epoch 1/200
60772/60772 [==============================] - 54s 891us/step - loss: 2.1144 - acc: 0.4464 - val_loss: 1.5786 - val_acc: 0.5299
Epoch 2/200
60772/60772 [==============================] - 51s 835us/step - loss: 0.9770 - acc: 0.6877 - val_loss: 2.4359 - val_acc: 0.4075
Epoch 3/200
60772/60772 [==============================] - 51s 831us/step - loss: 0.7668 - acc: 0.7436 - val_loss: 1.1808 - val_acc: 0.6351
Epoch 4/200
60772/60772 [==============================] - 50s 828us/step - loss: 0.6671 - acc: 0.7759 - val_loss: 1.4929 - val_acc: 0.5910
Epoch 5/200
60772/60772 [==============================] - 51s 832us/step - loss: 0.5975 - acc: 0.7960 - val_loss: 1.1269 - val_acc: 0.6736
Epoch 6/200
60772/60772 [==============================] - 50s 831us/step - loss: 0.5443 - acc: 0.8114 - val_loss: 1.0368 - val_acc: 0.6794
Epoch 7/200
60772/60772 [==============================] - 50s 830us/step - loss: 0.5047 - acc: 0.8260 - val_l

In [0]:
model.save("model/flyai-slr-model.h5")

In [0]:
t1 = time.time() 
test_X, test_X1, test_y = make_data(list_path="listfile/SLR1_list_test.txt", n_classes=100, train=False)
t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

In [78]:
model.evaluate(x=[test_X,test_X1],y=test_y)

19898/19898 [==============================] - 6s 323us/step


[1.9188286782905561, 0.8401346868672602]